In [183]:
import re

#### 1-Eliminate implication

In [519]:
# Initial setup
counter = 97
# Define the replace_char_at_index function
def index_char(str, indx, char_new):
    return str[:indx] + char_new + str[indx + 1:]

# Define the eliminate_implication function
def E_I(expr):
    for i in range(len(expr)):
        if expr[i] == '⇒':
            # Replace '⇒' with '∨'
            expr = index_char(expr, i, "∨")
            for j in range(i-1, -1, -1):
                if expr[j] == '(':
                    # Replace the character before '(' with '~'
                    expr = expr.replace(expr[j-1], "~" + expr[j-1])
                    break
    return expr


# Test the function
logical_expression = "∀x(P(x)⇒Q(x))⇒(∀x z(x)⇒∀x s(x))"
expression_without_double_negation = E_I(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)



Original expression: ∀x(P(x)⇒Q(x))⇒(∀x z(x)⇒∀x s(x))
Expression without double negation: ∀x(~P(x)∨~Q(x))∨(∀x ~z(x)∨∀x s(x))


####  3-Remove double-not.

In [440]:
def remove_doube(exp):
  result = []
  i = 0
  while i < len(exp):
    # Check for double negation (~~)
    if exp[i:i+2] == '~~':
      # Skip over the double negation
      i += 2
    else:
      # Append the character to the result
      result.append(exp[i])
      i += 1
  return ''.join(result)

# Example usage
logical_expression = "~~(P & Q) "
expression_without_double_negation = remove_doube(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)


Original expression: ['~', '~', ['&', 'P', 'Q']]
Expression without double negation: [['&', 'P', 'Q']]


#### 2-Move negation inward (Demorgan Law)

In [443]:
def if_quantis(exp):
    final_expression = ""
    counter = 0
    # Loop through each character in the expression
    while counter < len(exp):
        # check if the current position start with negation forall  or negation there exist 
        if exp.startswith("~∀", counter) or exp.startswith("~∃", counter):
            # Extract the quantifier type (opposite of the negated one)
            quanti = '∃' if exp[counter + 1] == '∀' else '∀'
            # Add the extracted quantifier to the final expression
            final_expression += quanti
            # Move the counter past the negation and quantifier symbols
            counter += 2
            # Add the next character (variable name) and a negation symbol
            variable = exp[counter]
            final_expression += variable + "~"
            counter += 1
        else:
            # If not a negated quantifier, simply add the current character
            final_expression += exp[counter]
            counter += 1
    return final_expression

def MNI(expression):
    if expression.find('~') == 2:
        expression = remove_doube(expression)
    # Apply quantifier manipulation
    expression = if_quantis(expression)
    # Move negation inward using De Morgan's Law
    # Define patterns for negation of AND and OR
    pat_and = r'~\((.*?)\s*&\s*(.*?)\)'
    pat_or = r'~\((.*?)\s*\|\s*(.*?)\)'
     # Apply De Morgan's Law for AND and OR
    expression = re.sub(pat_and, r'(~\1 | ~\2)', expression)
    expression = re.sub(pat_or, r'(~\1 & ~\2)', expression)
    
    # Check if the expression starts with a negation
    if expression.startswith('~'):
        # Remove the negation symbol (~) from the expression
        expression = expression[1:]
        # Negate the entire expression
        expression = "~(" + expression + ")"
    
    # Loop until all negations (~) have been moved inward
    while True:
        # Find the start index of the next negation (~)
        negation_start = expression.find('~(')
        # If no negation is found, exit the loop
        if negation_start == -1:
            break
        
        # Find the corresponding closing parenthesis for the negation
        open_paren_count = 1
        negation_end = negation_start + 2
        for i in range(negation_start + 2, len(expression)):
            if expression[i] == '(':
                open_paren_count += 1
            elif expression[i] == ')':
                open_paren_count -= 1
                if open_paren_count == 0:
                    negation_end = i
                    break
        
        # Extract the inner formula enclosed by the negation
        inner_formula = expression[negation_start + 2:negation_end]
        
        # Split the inner formula by the connective symbols (| or &)
        subExp = inner_formula.split('|') if '|' in inner_formula else inner_formula.split('&')
        
        # Negate each subformula and reconstruct the inner formula
        negated_subformulas = ['~' + subformula if subformula[0] != '~' else subformula[1:] for subformula in subExp]
        # Determine the new logical connective based on the original inner formula
        new_connective = '&' if '|' in inner_formula else '|'
        inner_formula = '(' + new_connective.join(negated_subformulas) + ')'
        
        # Replace the negated expression with the transformed inner formula
        expression = expression[:negation_start] + inner_formula + expression[negation_end + 1:]
    
    # Return the expression with all negations moved inward
    return expression

# Example usage:
formula = "ّّ~~∀x(~(P(x) & Q(x)) | ~R(x))"
formula1 = "~∀x(P(x) & Q(x))"
moved_formula = MNI(formula)
print("Original formula:", formula)
print("Formula with negation moved inward:", moved_formula)
print("=======================================")
moved_formula1 = MNI(formula1)
print("Original formula1:", formula1)
print("Formula with negation moved inward:", moved_formula1)
print("=======================================")
formula2 = "~∃x(~P(x) | Q(x))"
moved_formula2 = MNI(formula2)
print("Original formula2:", formula2)
print("Formula with negation moved inward:", moved_formula2)
print("=======================================")
formula3 = "~∃x∀y(P(x, y) & Q(x, y))"
moved_formula3 = MNI(formula3)
print("Original formula3:", formula3)
print("Formula with negation moved inward:", moved_formula3)
print("=======================================")




Original expression: =>&PQ=>~R|ST
Expression after Multi-Not Inward (MNI) transformation: =>&PQ=>~R|ST


####  4-Standardize variable scope

In [452]:

def SVS(exp):
    result = ""
    for expr in exp:
        new_expr = expr
        vars_seen = set()
        rep_map = {}
        
        def var_new(var_old):
            if var_old in rep_map:
                return rep_map[var_old]
            new_var = chr(ord('a') + len(vars_seen))
            while new_var in vars_seen:
                new_var = chr(ord(new_var) + 1)
            vars_seen.add(new_var)
            rep_map[var_old] = new_var
            return new_var

        counter = 0
        while counter < len(new_expr):
            if new_expr[counter] in "∀∃":
                if counter + 1 < len(new_expr):
                    var = new_expr[counter + 1]
                    if var.islower():  
                        new_var = var_new(var)
                        new_expr = new_expr[:counter + 1] + new_expr[counter + 1:].replace(var, new_var)
                        counter += len(new_var) - 1
            counter += 1
        result += new_expr   # Concatenate the processed expression with a space
        
    return result.strip()  # Remove trailing space before returning

# Test case

expressions_1 = ['∃x(P(x) & Q(x)) '  ' ∀x(R(x) | S(x))']
print("Original Expressions 1:", expressions_1)
print("SVS Result 1:", SVS(expressions_1))


# Test case 3
expressions_2 = ['∃x∀y(P(x, y) & Q(x, y))' '∀z∃w(R(z, w) | S(z, w))']
print("\nOriginal Expressions 3:", expressions_2)
print("SVS Result 3:", SVS(expressions_2))


Original Expressions 1: ['∃x(P(x) & Q(x))  ∀x(R(x) | S(x))']
SVS Result 1: ∃a(P(a) & Q(a))  ∀b(R(b) | S(b))

Original Expressions 3: ['∃x∀y(P(x, y) & Q(x, y))∀z∃w(R(z, w) | S(z, w))']
SVS Result 3: ∃a∀b(P(a, b) & Q(a, b))∀c∃d(R(c, d) | S(c, d))


#### 5- The prenex form (obtained by moving all quantifiers to the left of the
formula.)

In [66]:
#done
def penex_form(exp):
    quanti_part = ""
    parts = ""
    i = 0
    while i < len(exp):
        if exp[i] == '∀' or exp[i] == '∃':
            quanti_part += exp[i] + exp[i+1] + ' '
            i += 1  # Skip the next character which is the variable
        else:
            parts += exp[i]
        i += 1
    return quanti_part.strip() + " " + parts.strip()

# Test case
expression = '∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))'
print("Original Expression:", expression)
print("Penex Form:", penex_form(expression))


Original Expression: ∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))
Penex Form: ∃x ∀y (P(x) ∧ Q(x)) ∧ (R(y) ∨ S(y))


#### 6-Skolemization for existential quantifiers

In [226]:
def skolemization(exp):
    part = exp.split('∃')
    parts_skol = [part[0]]
    for part in part[1:]:
        if part.strip():
            var, i, exp_in = part.partition('(')
            var += '('  # Adding back the opening parenthesis
            
            # Create Skolem function with parentheses
            func_skol = "f" + "("+var.replace('(', '').replace(')', ')')  + ")" # Add closing parenthesis
            
            # Replace variable x in the inner expression with the Skolem function
            skolemized_inner_exp = exp_in.replace('x',  func_skol )
            
            
            # Append skolemized inner expression to the list of skolemized parts
            parts_skol.append(skolemized_inner_exp)
    return ''.join(parts_skol)

# Example usage:
logical_expression = "∃x (P(x) & Q(x))"
skolemized_expression = skolemization(logical_expression)
print("Original expression:", logical_expression)
print("Skolemized expression:", skolemized_expression)


Original expression: ∃x (P(x) & Q(x))
Skolemized expression: P(f(x )) & Q(f(x )))


#### 7-Eliminate universal quantifiers.

In [299]:
# done
def E_U(exp):
    # Define a pattern to match universal quantifiers and their corresponding variables and inner expressions
    patrn = r'∀(\w+)\s*\((.*?)\)'

    # Use regex to remove all occurrences of universal quantifiers
    withoutForAll = re.sub(patrn, r'(\2)', exp)

    return withoutForAll

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
expression_without_universal_quantifiers = E_U(logical_expression)
print("Original expression:", logical_expression)
print("Expression without universal quantifiers:", expression_without_universal_quantifiers)


Original expression: ∀x (P(x) & Q(x))
Expression without universal quantifiers: (P(x) & Q(x))


#### 8-Convert to conjunctive normal form

#### - eliminate_bidirectional

In [514]:
def E_b(exp):
    # Define a pattern to match bidirectional implications
    pat = r'(\([^()]+\)|\b\w+\b(?:\([^()]+\))?)\s*<=>\s*(\([^()]+\)|\b\w+\b(?:\([^()]+\))?)'

    # Repeat until no more bidirectional implications
    while re.search(pat, exp):
        # Find bidirectional implications
        for i in re.finditer(pat, exp):
            left_part, right_part = i.groups()

            # Replace bidirectional implication with conjunction of implications
            exp = exp[:i.start()] + f'({left_part} -> {right_part}) & ({right_part} -> {left_part})' + exp[i.end():]

    return exp

# Test case
test_cases = [
    "(P <=> Q)",
    "(A <=> B) & (B <=> C)",
    "(X <=> Y) & (Y <=> Z)",
    "(A <=> (B <=> C))",
    "(P(x) <=> Q(y)) & (R(z) <=> S(w))"
]

print("Original expressions:")
for test_case in test_cases:
    print(test_case)

print("\nExpressions after bidirectional elimination:")
for test_case in test_cases:
    result = E_b(test_case)
    print(result)



Original expressions:
(P <=> Q)
(A <=> B) & (B <=> C)
(X <=> Y) & (Y <=> Z)
(A <=> (B <=> C))
(P(x) <=> Q(y)) & (R(z) <=> S(w))

Expressions after bidirectional elimination:
((P -> Q) & (Q -> P))
((A -> B) & ((B -> C) & (C -> B)) & ((B -> C) & (C -> B))
((X -> Y) & ((Y -> Z) & (Z -> Y)) & ((Y -> Z) & (Z -> Y))
((A -> ((B -> C) & (C (B -> C) & (C -> B) & (((B -> C) & (C -> B)) -> A))
((P(x) -> Q(y)) & ((R(z) -> S(w)) & (S(w) -> R(z))) & ((R(z) -> S(w)) & (S(w) -> R(z)))


In [297]:
def to_conj_of_disj(expression):
    # Split the expression by '&' to get individual conjuncts
    conjuncts = expression.split('&')
    
    # Initialize the result as an empty string
    result = ""
    
    # Iterate through each conjunct
    for conjunct in conjuncts:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '|' in conjunct:
            disjunctions = conjunct.split('|')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjunctions) > 1:
                conjunct = '(' + conjunct + ')'
        
        # Append the modified conjunct to the result with '&' in between
        result += conjunct + ' & '
    
    # Remove the trailing '&' and return the result
    return result[:-3]  # Remove the last ' & '

# Test case for to_conj_of_disj function
expression = "(p & q) | r"
result = to_conj_of_disj(expression)
print("Original expression:", expression)
print("Expression in conjunctive form:", result)
print("===========================================")
# Test case 2
expression_2 = "(p & q) | (r | s)"
result_2 = to_conj_of_disj(expression_2)
print("\nOriginal expression 2:", expression_2)
print("Expression in conjunctive form 2:", result_2)
print("===========================================")
# Test case 3
expression_3 = "(p & q) | (r & s)"
result_3 = to_conj_of_disj(expression_3)
print("\nOriginal expression 3:", expression_3)
print("Expression in conjunctive form 3:", result_3)
# Test case 4
print("===========================================")
expression_4 = "p & (q | r) & (s | t)"
result_4 = to_conj_of_disj(expression_4)
print("\nOriginal expression 4:", expression_4)
print("Expression in conjunctive form 4:", result_4)

Original expression: (p & q) | r
Expression in conjunctive form: (p  & ( q) | r)

Original expression 2: (p & q) | (r | s)
Expression in conjunctive form 2: (p  & ( q) | (r | s))

Original expression 3: (p & q) | (r & s)
Expression in conjunctive form 3: (p  & ( q) | (r ) &  s)

Original expression 4: p & (q | r) & (s | t)
Expression in conjunctive form 4: p  & ( (q | r) ) & ( (s | t))


In [515]:
def convert_to_CNF(expression):
    print("Original Expression:", expression)
    
    # Step 1: Eliminate BIDIRECTIONAL
    expression = E_b(expression)
    print("After Step 1 (Eliminate BIDIRECTIONAL):", expression)
    
    # Step 2: Eliminate Implications
    expression = eliminate_implication(expression)
    print("After Step 2 (Eliminate Implications):", expression)
    
    # Step 3: Move Negation Inward
    expression = MNI(expression)
    print("After Step 3 (Move Negation Inward):", expression)
    
    # Step 4: Remove Double Negations
    expression = remove_doube(expression)
    print("After Step 4 (Remove Double Negations):", expression)
    
    # Step 5: Standardize Variable Scope
    expression = [expression]
    
    expression = SVS(expression)
    print("After Step 5 (Standardize Variable Scope):", expression)
    # Step 6: Move Universal Quantifiers Outward
    expression = E_U(expression)
    print("After Step 6 (Move Universal Quantifiers Outward):", expression)
    
    # Step 7: Skolemization
    expression = skolemization(expression)
    print("After Step 7 (Skolemization):", expression)
    
    # Step 8: Convert to CNF
    expression = to_conj_of_disj(expression)
    print("After Step 8 (Convert to CNF):", expression)
    
    return expression


logical_expressions = {
    "Logical Expression 1": "∀x(P(x)⇒Q(x))",
    "Logical Expression 2": "(P(x)<=>Q(y))"
}

cnf_expressions = {}
for name, expression in logical_expressions.items():
    cnf_expression = convert_to_CNF(expression)
    cnf_expressions[name] = cnf_expression
    print("===============================================")

print("\nSet of CNF Expressions:")
for name, expression in cnf_expressions.items():
    print(name, ":", expression)





Original Expression: ∀x(P(x)⇒Q(x))
After Step 1 (Eliminate BIDIRECTIONAL): ∀x(P(x)⇒Q(x))
After Step 2 (Eliminate Implications): ∀x(~P(x)∨Q(x))
After Step 3 (Move Negation Inward): ∀x(~P(x)∨Q(x))
After Step 4 (Remove Double Negations): ∀x(~P(x)∨Q(x))
After Step 5 (Standardize Variable Scope): ∀a(~P(a)∨Q(a))
After Step 6 (Move Universal Quantifiers Outward): (~P(a)∨Q(a))
After Step 7 (Skolemization): (~P(a)∨Q(a))
After Step 8 (Convert to CNF): (~P(a)∨Q(a))
Original Expression: (P(x)<=>Q(y))
After Step 1 (Eliminate BIDIRECTIONAL): ((P(x) -> Q(y)) & (Q(y) -> P(x)))
After Step 2 (Eliminate Implications): ((P(x) -> Q(y)) & (Q(y) -> P(x)))
After Step 3 (Move Negation Inward): ((P(x) -> Q(y)) & (Q(y) -> P(x)))
After Step 4 (Remove Double Negations): ((P(x) -> Q(y)) & (Q(y) -> P(x)))
After Step 5 (Standardize Variable Scope): ((P(x) -> Q(y)) & (Q(y) -> P(x)))
After Step 6 (Move Universal Quantifiers Outward): ((P(x) -> Q(y)) & (Q(y) -> P(x)))
After Step 7 (Skolemization): ((P(x) -> Q(y)) & (Q(y

#### Step 9: Turn Conjunctions into Clauses

In [523]:
def turn_into_clauses(expression):
    # Split the expression by '&' to get individual conjuncts
    conj = expression.split('&')
    clause = []
    # Iterate through each conjunct
    for conjunct in conj:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '|' in conjunct:
            disjun = conjunct.split('|')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjun) > 1:
                conjunct = '(' + conjunct + ')'
       
        # Append the modified conjunct to the list of clauses
        clause.append(conjunct.strip())
    
    return set(clause)  # Using set to ensure uniqueness of clauses
#test cases 
test_cases = [
    "(P(x) -> Q(y)) & (Q(y) -> P(x))",  
    "(A & B) & (C | D)",  
    "A & B & C & D",  
    "(P(x) | Q(y)) & (R(z))", 
    "A"  
]

# Test the function with each test case
for i, test_case in enumerate(test_cases, 1):
    print(f"Test Case {i}: {test_case}")
    result = turn_into_clauses(test_case)
    print("Clauses:")
    for clause in result:
        print(clause)
    print()


Test Case 1: (P(x) -> Q(y)) & (Q(y) -> P(x))
Clauses:
(P(x) -> Q(y))
(Q(y) -> P(x))

Test Case 2: (A & B) & (C | D)
Clauses:
(A
B)
( (C | D))

Test Case 3: A & B & C & D
Clauses:
A
D
C
B

Test Case 4: (P(x) | Q(y)) & (R(z))
Clauses:
((P(x) | Q(y)) )
(R(z))

Test Case 5: A
Clauses:
A



##### 10.Rename variables in clauses so that each clause has a unique variable name

In [526]:
def split_cnf_and_rename_variables(expression):
    """Split CNF expression into clauses and rename variables in each clause to ensure local uniqueness."""
    clauses = expression.split(" & ")
    return further_rename_variables(clauses)

def further_rename_variables(clauses):
    """Renames variables in clauses to ensure unique names across all clauses."""
    # Track used variable names
    used_variables = set()
    counter = 0

    # Rename variables in each clause
    renamed_clauses = set()
    for clause in clauses:
        new_clause = ""
        for literal in clause.split(" | "):
            if literal[0] in "PQRSTUVWXYZ":
                new_var = f"x{counter}"
                while new_var in used_variables:  # Check for conflicts
                    counter += 1
                    new_var = f"x{counter}"
                used_variables.add(new_var)  # Add to used variables
                new_clause += new_var if literal[0] != "~" else f"~{new_var}"
            else:
                new_clause += literal
            new_clause += " | "
        renamed_clauses.add(new_clause[:-2])  # Remove trailing "|"

    return renamed_clauses

# Test case for splitting CNF and renaming variables

# Formula with potential conflicts across clauses
formula = "(P | P) & (Q | Q) & (R | S)"

# Split and rename variables within each clause (step 9)
clauses_with_local_renaming = split_cnf_and_rename_variables(formula)
print("Clauses with local renaming:", clauses_with_local_renaming)

# Formula with potential conflicts across all clauses
formula2 = "(~x | ~x) & (y | y) & (z | w)"

# Split and rename variables within each clause (step 9)
clauses_with_local_renaming2 = split_cnf_and_rename_variables(formula2)
print("Clauses with local renaming (formula 2):", clauses_with_local_renaming2)

# Further rename variables for global uniqueness (step 10)
globally_unique_clauses = further_rename_variables(clauses_with_local_renaming)
print("Clauses with globally unique variables:", globally_unique_clauses)

# Further rename variables for formula 2 (optional)
globally_unique_clauses2 = further_rename_variables(clauses_with_local_renaming2)
print("Clauses with globally unique variables (formula 2):", globally_unique_clauses2)


Clauses with local renaming: {'(R | x2 ', '(Q | x1 ', '(P | x0 '}
Clauses with local renaming (formula 2): {'(y | y) ', '(~x | ~x) ', '(z | w) '}
Clauses with globally unique variables: {'(P | x0  ', '(Q | x1  ', '(R | x2  '}
Clauses with globally unique variables (formula 2): {'(~x | ~x)  ', '(y | y)  ', '(z | w)  '}


In [524]:
def further_rename_variables(clauses):
  """Renames variables in clauses to ensure unique names across all clauses.

  Args:
      clauses: A set of strings representing clauses in CNF.

  Returns:
      A set of strings representing clauses with unique variables across all clauses.
  """

  # Track used variable names
  used_variables = set()
  counter = 0

  # Rename variables in each clause
  renamed_clauses = set()
  for clause in clauses:
    new_clause = ""
    for literal in clause.split(" | "):
      if literal[0] in "PQRSTUVWXYZ":
        new_var = f"x{counter}"
        while new_var in used_variables:  # Check for conflicts
          counter += 1
          new_var = f"x{counter}"
        used_variables.add(new_var)  # Add to used variables
        new_clause += new_var if literal[0] != "~" else f"~{new_var}"
      else:
        new_clause += literal
      new_clause += " | "
    renamed_clauses.add(new_clause[:-2])  # Remove trailing "|"

  return renamed_clauses
# Test case for splitting CNF and renaming variables

# Formula with potential conflicts across clauses
formula = "(P | P) & (Q | Q) & (R | S)"

# Split and rename variables within each clause (step 9)
clauses_with_local_renaming = split_cnf_and_rename_variables(formula)
print("Clauses with local renaming:", clauses_with_local_renaming)

# Formula with potential conflicts across all clauses
formula2 = "(~x | ~x) & (y | y) & (z | w)"

# Split and rename variables within each clause (step 9)
clauses_with_local_renaming2 = split_cnf_and_rename_variables(formula2)
print("Clauses with local renaming (formula 2):", clauses_with_local_renaming2)

# Further rename variables for global uniqueness (step 10)
globally_unique_clauses = further_rename_variables(clauses_with_local_renaming)
print("Clauses with globally unique variables:", globally_unique_clauses)

# Further rename variables for formula 2 (optional)
globally_unique_clauses2 = further_rename_variables(clauses_with_local_renaming2)
print("Clauses with globally unique variables (formula 2):", globally_unique_clauses2)


NameError: name 'split_cnf_and_rename_variables' is not defined

#### Resolution function 

In [527]:
# Apply Resolution Procedure
def resolution_procedure(expression):
    # Apply each step of resolution procedure
    expression = E_I(expression)
    expression = MNI(expression)
    expression = remove_doube(expression)
    expression = SVS(expression)
    expression = penex_form(expression)
    expression = skolemization(expression)
    expression = E_U(expression)
    expression = convert_to_CNF(expression)
    #note complete 
    expression = turn_into_clauses(expression)
    # expression = rename_variables_in_clauses(expression)
    # Perform resolution here
    
    return expression
# Test case for resolution_procedure function
test_expression = "((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))"

result = resolution_procedure(test_expression)
print("Result after resolution procedure:", result)


Original Expression:  ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 1 (Eliminate BIDIRECTIONAL):  ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 2 (Eliminate Implications):  ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 3 (Move Negation Inward):  ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 4 (Remove Double Negations):  ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 5 (Standardize Variable Scope): ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 6 (Move Universal Quantifiers Outward): ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 7 (Skolemization): ((P(x) & Q(x)) | (~P(x) & R(x))) => ((~Q(x) & S(x)) | (~R(x) & T(x)))
After Step 8 (Convert to CNF): ((P(x)  & ( Q(x)) | (~P(x) ) &  R(x))) => ((~Q(x)  & ( S(x)) | (~R(x) ) &  T(x)))
Result after resoluti

In [529]:
import re

def resolution(expression):
    # Initialize counters for positive and negative occurrences of words
    positive_counter = {}
    negative_counter = {}

    # Function to update counters based on the occurrence of words
    def update_counters(match):
        word = match.group(1)
        sign = match.group(2)

        if sign == '':
            # Positive occurrence
            positive_counter[word] = positive_counter.get(word, 0) + 1
        else:
            # Negative occurrence
            negative_counter[word] = negative_counter.get(word, 0) + 1

    # Update counters by finding words and their signs in the expression
    re.sub(r'\b(\w+)\b(\')?', update_counters, expression)

    # Remove words appearing both positively and negatively
    for word in positive_counter.keys() & negative_counter.keys():
        count = min(positive_counter[word], negative_counter[word])
        del positive_counter[word]
        del negative_counter[word]
        # Decrement the counters
        positive_counter[word] -= count
        negative_counter[word] -= count

    # Construct the resulting expression with updated counters
    result_expression = ''
    for word, count in positive_counter.items():
        if count > 0:
            result_expression += f"{word} " * count
    for word, count in negative_counter.items():
        if count > 0:
            result_expression += f"~{word} " * count

    return result_expression.strip()


# Test case
test_expression = "P(x) & Q(x) & ~P(x) & R(x) & ~Q(x)"
result = resolution(test_expression)
print("Result after resolution:", result)


Result after resolution: ~P ~P ~x ~x ~x ~x ~x ~Q ~Q ~R


In [530]:
def resolution(expression):
    # Function to count occurrences of each word
    def count_words(expr):
        words = expr.split()
        pos_counter = {}
        neg_counter = {}

        for word in words:
            if word[0] == '~':  # Check if it's a negated word
                word = word[1:]
                neg_counter[word] = neg_counter.get(word, 0) + 1
            else:
                pos_counter[word] = pos_counter.get(word, 0) + 1

        return pos_counter, neg_counter

    # Function to remove words appearing in both positive and negative forms
    def remove_duplicates(pos_counter, neg_counter):
        duplicates = set(pos_counter.keys()) & set(neg_counter.keys())

        for word in duplicates:
            min_count = min(pos_counter[word], neg_counter[word])
            if min_count > 0:
                pos_counter[word] -= min_count
                neg_counter[word] -= min_count

        return pos_counter, neg_counter

    # Apply resolution procedure
    pos_counter, neg_counter = count_words(expression)
    pos_counter, neg_counter = remove_duplicates(pos_counter, neg_counter)

    # Check if any counter becomes zero
    for word, count in pos_counter.items():
        if count == 0:
            return None
    for word, count in neg_counter.items():
        if count == 0:
            return None

    # Reconstruct expression from modified counters
    modified_expression = ''
    for word, count in pos_counter.items():
        modified_expression += (word + ' ') * count
    for word, count in neg_counter.items():
        modified_expression += (('~' + word + ' ') * count)

    return modified_expression.strip()

# Test case
test_expression = "~P(x) & Q(x) | ~P(x) | R(x) | S(x)"
result = resolution(test_expression)
print("Result:", result)


Result: & Q(x) | | | R(x) S(x) ~P(x) ~P(x)
